![](./images/Title.PNG)
<div class="alert alert-block alert-info"> <b> </b> 
</div>

# Fraud detection from transactions
![](./images/workflow_fraud.PNG)

<div class="alert alert-block alert-info"> <b> </b> 
</div>

## 1 - Connect to Vantage
<div class="alert alert-block alert-info"> <b> </b> 
</div>

![](./images/Slide32.PNG)

In [1]:
import teradataml as tdml
tdml.options.configure.byom_install_location = "mldb"
tdml.display.print_sqlmr_query = False
import getpass
import json
tdml.__version__
from datetime import datetime
tic = datetime.now()
tdml.options.display
import sys

In [2]:
tdml.__version__

'17.20.00.04'

In [3]:
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import os

In [4]:
# set the path to the local project repository for this model demo
model_local_path = 'C:/Users/dm250067/OneDrive - Teradata/Documents/01 - Code Development/modelops-demo-models/model_definitions/transaction_fraud_indb'
res = os.system(f'mkdir -p "{model_local_path}"')
res = os.system(f'mkdir -p "{model_local_path}/model_modules"')

In [5]:
Param = {
    'host'          : 'tdprd2.td.teradata.com', 
    'user'          : 'dm250067', 
    'password'      : "ENCRYPTED_PASSWORD(file:{},file:{})".format ('../../PassKey.properties','../../EncPass.properties'), #getpass.getpass(), 
    'logmech'       : 'LDAP',
    'database'      : 'ADLSLSEMEA_DEMO_BANKING',
    'temp_database_name' : 'dm250067'
    }

Param = {
    'host'          : 'tdprd3.td.teradata.com', 
    'user'          : 'dm250067', 
    'password'      : "ENCRYPTED_PASSWORD(file:{},file:{})".format ('../../PassKey.properties','../../EncPass.properties'), #getpass.getpass(), 
    'logmech'       : 'LDAP',
    'database'      : 'ADLDSD_CHURN',
    'temp_database_name' : 'dm250067'
    }

tdml.create_context(**Param)

C:\Users\dm250067\Anaconda3\envs\vantage39\lib\site-packages\teradataml\context\context.py:480: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)


Engine(teradatasql://:***@tdprd3.td.teradata.com/?DATABASE=ADLDSD_CHURN&LOGDATA=%2A%2A%2A&LOGMECH=%2A%2A%2A&USER=DM250067)

## 2 - Define Training Function

In [10]:
%%writefile "$model_local_path/model_modules/training.py"
from teradataml import (
    DataFrame,
    OneHotEncodingFit,
    OneHotEncodingTransform,
    ScaleFit,
    ScaleTransform,
    DecisionForest,
    configure
)
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import numpy as np

configure.val_install_location = 'TRNG_XSP'

def plot_roc_curve(fi, img_filename):
    import pandas as pd
    import matplotlib.pyplot as plt
    feat_importances = pd.Series(fi)
    feat_importances.nlargest(10).plot(kind='barh').set_title('Feature Importance')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()


def train(context: ModelContext, **kwargs):
    aoa_create_context()

    feature_names = context.dataset_info.feature_names
    target_name   = context.dataset_info.target_names[0]
    entity_key    = context.dataset_info.entity_key

    # read training dataset from Teradata and convert to pandas
    train_df      = DataFrame.from_query(context.dataset_info.sql)
    
    print('feature names :', feature_names)
    
    if 'transaction_type' in feature_names:
        print ("OneHotEncoding using InDB Functions...")
        
        transaction_types = list(train_df[['transaction_type','txn_id']].groupby(['transaction_type']).count().to_pandas()['transaction_type'].values)


        onehot = OneHotEncodingFit(data           = train_df,
                                        is_input_dense  = True,
                                        target_column      = 'transaction_type',
                                        categorical_values = transaction_types,
                                        other_column="other"
                                       )

        train_df_onehot = OneHotEncodingTransform(data=train_df,
                                           object=onehot.result,
                                           is_input_dense=True
                                          ).result

        onehot.result.to_sql(f"onehot_${context.model_version}", if_exists="replace")
        print("Saved onehot")
        
        feature_names_after_one_hot = [c for c in feature_names if c != 'transaction_type'] + ['transaction_type_'+c for c in transaction_types]
        category_features = ['transaction_type']
    else:
        train_df_onehot = train_df
        feature_names_after_one_hot = feature_names
        category_features = []
    
    print ("Scaling using InDB Functions...")
    print(feature_names_after_one_hot)
    
    scaler = ScaleFit(
        data           = train_df_onehot,
        target_columns = feature_names_after_one_hot,
        scale_method   = context.hyperparams["scale_method"],
        miss_value     = context.hyperparams["miss_value"],
        global_scale   = context.hyperparams["global_scale"].lower() in ['true', '1'],
        multiplier     = context.hyperparams["multiplier"],
        intercept      = context.hyperparams["intercept"]
    )

    scaled_train = ScaleTransform(
        data           = train_df_onehot,
        object         = scaler.output,
        accumulate     = [target_name, entity_key]
    ).result
    
    scaler.output.to_sql(f"scaler_${context.model_version}", if_exists="replace")
    print("Saved scaler")
    
    print("Starting training...")
    model = DecisionForest(
        input_columns        = feature_names_after_one_hot,
        response_column      = target_name,
        data                 = scaled_train,
        max_depth            = context.hyperparams["max_depth"],
        num_trees            = context.hyperparams["num_trees"],
        min_node_size        = context.hyperparams["min_node_size"],
        mtry                 = context.hyperparams["mtry"],
        mtry_seed            = context.hyperparams["mtry_seed"],
        seed                 = context.hyperparams["seed"],
        tree_type            = 'CLASSIFICATION'
    )
    
    model.result.to_sql(f"model_${context.model_version}", if_exists="replace")    
    print("Saved trained model")

    record_training_stats(
        train_df,
        features    = feature_names,
        targets     = [target_name],
        categorical = [target_name]+category_features,
        #feature_importance = {f:0 for f in feature_names_after_one_hot},
        context     = context
    )

Overwriting C:/Users/dm250067/OneDrive - Teradata/Documents/01 - Code Development/modelops-demo-models/model_definitions/transaction_fraud_indb/model_modules/training.py


In [7]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the training dataset 
sql = f"""
SELECT 
*
FROM {Param['database']}.transactions_fraud
where fold = 'train'
"""

feature_metadata =  {
    "database": Param['database'],
    "table": "transactions_features"
}

hyperparams = {
    # scaler
    "scale_method":"STD",
    "miss_value":"KEEP",
    "global_scale":"False",
    "multiplier":"1",
    "intercept":"0",
    # decision forest
    "max_depth": 15, 
    "num_trees": 72,
    "min_node_size": 1,
    "mtry": 6,
    "mtry_seed": 1,
    "seed": 1,
}

entity_key    = "txn_id"
target_names  = ["isFraud"]
feature_names = ['amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'transaction_type']

from aoa import ModelContext, DatasetInfo

dataset_info = DatasetInfo(
    sql=sql,
    entity_key=entity_key,
    feature_names=feature_names,
    target_names=target_names,
    feature_metadata=feature_metadata
)

ctx = ModelContext(
    hyperparams=hyperparams,
    dataset_info=dataset_info,
    artifact_output_path=f'{model_local_path}/model_modules/artifacts/',
    model_version="InDB_v1",
    model_table="aoa_model_indb_v1"
)

sys.path.append(os.path.expanduser(f"{model_local_path}/model_modules"))
import training
training.train(context=ctx)

feature names : ['amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'transaction_type']
OneHotEncoding using InDB Functions...
Saved onehot
Scaling using InDB Functions...
['amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'transaction_type_CASH_IN', 'transaction_type_CASH_OUT', 'transaction_type_TRANSFER', 'transaction_type_PAYMENT', 'transaction_type_DEBIT']
Saved scaler
Starting training...
Saved trained model


## 3 - Define Evaluation Function

In [8]:
%%writefile "$model_local_path/model_modules/evaluation.py"
from sklearn.metrics import confusion_matrix
from teradataml import (
    copy_to_sql,
    DataFrame,
    OneHotEncodingTransform,
    DecisionForestPredict,
    TDGLMPredict,
    ScaleTransform,
    ClassificationEvaluator,
    ConvertTo,
    ROC
)
from aoa import (
    record_evaluation_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import json
import os
import numpy as np


def plot_feature_importance(fi, img_filename):
    import pandas as pd
    import matplotlib.pyplot as plt
    feat_importances = pd.Series(fi)
    feat_importances.nlargest(10).plot(kind='barh').set_title('Feature Importance')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()
    
    
def plot_confusion_matrix(cf, img_filename):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(cf, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(cf.shape[0]):
        for j in range(cf.shape[1]):
            ax.text(x=j, y=i,s=cf[i, j], va='center', ha='center', size='xx-large')
    ax.set_xlabel('Predicted labels');
    ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix');
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()

    
def plot_roc_curve(roc_out, img_filename):
    import matplotlib.pyplot as plt
    auc = roc_out.result.to_pandas().reset_index()['AUC'][0]
    roc_results = roc_out.output_data.to_pandas()
    plt.plot(roc_results['fpr'], roc_results['tpr'], color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % 0.27)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()

    
def evaluate(context: ModelContext, **kwargs):

    aoa_create_context()

    model = DataFrame(f"model_${context.model_version}")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    test_df = DataFrame.from_query(context.dataset_info.sql)
    
    # One hot encoding
    if 'transaction_type' in feature_names:
        print ("Loading onehot...")
        onehot = DataFrame(f"onehot_${context.model_version}")
        onehot_test = OneHotEncodingTransform(data=test_df,
                              object=onehot,
                              is_input_dense=True).result        
    else:
        print ("no onehotencoding")
        onehot_test = test_df
    

    # Scaling the test set
    print ("Loading scaler...")
    scaler = DataFrame(f"scaler_${context.model_version}")

    scaled_test = ScaleTransform(
        data=onehot_test,
        object=scaler,
        accumulate = [target_name,entity_key]
    ).result
    
    print("Scoring")
    predictions = DecisionForestPredict(object = model,
                                                newdata = scaled_test,
                                                id_column = entity_key,                                                        
                                                terms = [target_name],
                                                accumulate = [target_name],
                                                output_prob = True,
                                                output_responses=["0","1"]
                                                )
    print(predictions)

    predicted_data = ConvertTo(
        data = predictions.result,
        target_columns = [target_name,'prediction'],
        target_datatype = ["INTEGER"]
    )

    ClassificationEvaluator_obj = ClassificationEvaluator(
        data=predicted_data.result,
        observation_column=target_name,
        prediction_column='prediction',
        num_labels=2
    )

    metrics_pd = ClassificationEvaluator_obj.output_data.to_pandas()

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics_pd.MetricValue[0]),
        'Micro-Precision': '{:.2f}'.format(metrics_pd.MetricValue[1]),
        'Micro-Recall': '{:.2f}'.format(metrics_pd.MetricValue[2]),
        'Micro-F1': '{:.2f}'.format(metrics_pd.MetricValue[3]),
        'Macro-Precision': '{:.2f}'.format(metrics_pd.MetricValue[4]),
        'Macro-Recall': '{:.2f}'.format(metrics_pd.MetricValue[5]),
        'Macro-F1': '{:.2f}'.format(metrics_pd.MetricValue[6]),
        'Weighted-Precision': '{:.2f}'.format(metrics_pd.MetricValue[7]),
        'Weighted-Recall': '{:.2f}'.format(metrics_pd.MetricValue[8]),
        'Weighted-F1': '{:.2f}'.format(metrics_pd.MetricValue[9]),
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)
        
    cm = confusion_matrix(predicted_data.result.to_pandas()['isFraud'], predicted_data.result.to_pandas()['prediction'])
    plot_confusion_matrix(cm, f"{context.artifact_output_path}/confusion_matrix")

    roc_out = ROC(
        data=predictions.result,
        probability_column='prob',
        observation_column=target_name,
        positive_class='1',
        num_thresholds=1000
    )
    plot_roc_curve(roc_out, f"{context.artifact_output_path}/roc_curve")

    predictions_table = "predictions_tmp"
    copy_to_sql(df=predicted_data.result, table_name=predictions_table, index=False, if_exists="replace", temporary=True)

    # calculate stats if training stats exist
    if os.path.exists(f"{context.artifact_input_path}/data_stats.json"):
        record_evaluation_stats(
            features_df=test_df,
            predicted_df=DataFrame.from_query(f"SELECT * FROM {predictions_table}"),
            #feature_importance=feature_importance,
            context=context
        )

Overwriting C:/Users/dm250067/OneDrive - Teradata/Documents/01 - Code Development/modelops-demo-models/model_definitions/transaction_fraud_indb/model_modules/evaluation.py


In [9]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = f"""
SELECT 
*
FROM {Param['database']}.transactions_fraud
where fold = 'test'
"""

entity_key    = "txn_id"
target_names  = ["isFraud"]
feature_names = ['amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'transaction_type']
 
dataset_info = DatasetInfo(
    sql=sql,
    entity_key=entity_key,
    feature_names=feature_names,
    target_names=target_names,
    feature_metadata=feature_metadata
)

ctx = ModelContext(
    hyperparams=hyperparams,
    dataset_info=dataset_info,
    artifact_output_path=f'{model_local_path}/model_modules/artifacts/',
    artifact_input_path=f'{model_local_path}/model_modules/artifacts/',
    model_version="InDB_v1",
    model_table="aoa_model_indb_v1"
)

import evaluation
evaluation.evaluate(context=ctx)
#evaluate(context=ctx)

# view evaluation results
import json
with open(f"{ctx.artifact_output_path}/metrics.json") as f:
    print(json.load(f))

Loading onehot...
Loading scaler...
Scoring


TeradataMlException: [Teradata][teradataml](TDML_2001) Following required arguments are missing: ['newdata'].

## 4 - Define Scoring Function

In [ ]:
%%writefile "$model_local_path/model_modules/scoring.py"
from teradataml import (
    copy_to_sql,
    DataFrame,
    TDGLMPredict,
    ScaleTransform,
    OneHotEncodingTransform,
    TDDecisionForestPredict
)
from aoa import (
    record_scoring_stats,
    aoa_create_context,
    ModelContext
)
import pandas as pd


def score(context: ModelContext, **kwargs):

    aoa_create_context()

    model = DataFrame(f"model_${context.model_version}")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    features_tdf = DataFrame.from_query(context.dataset_info.sql)
    features_pdf = features_tdf.to_pandas(all_rows=True)

    # One hot encoding
    if 'transaction_type' in feature_names:
        print ("Loading onehot...")
        onehot = DataFrame(f"onehot_${context.model_version}")
        onehot_test = OneHotEncodingTransform(data=features_tdf,
                              object=onehot,
                              is_input_dense=True).result        
    else:
        print ("no onehotencoding")
        onehot_test = features_tdf
    
    # Scaling the scoring set
    print ("Loading scaler...")
    scaler = DataFrame(f"scaler_${context.model_version}")

    scaled_features = ScaleTransform(
        data=onehot_test,
        object=scaler,
        accumulate = entity_key
    )
    
    print("Scoring")
    predictions = TDDecisionForestPredict(
        object=model,
        data=scaled_features.result,
        id_column=entity_key
    )

    predictions_pdf = predictions.result.to_pandas(all_rows=True).rename(columns={"prediction": target_name}).astype(int)

    print("Finished Scoring")

    # store the predictions
    predictions_pdf = pd.DataFrame(predictions_pdf, columns=[target_name])
    predictions_pdf[entity_key] = features_pdf.index.values
    # add job_id column so we know which execution this is from if appended to predictions table
    predictions_pdf["job_id"] = context.job_id

    # teradataml doesn't match column names on append.. and so to match / use same table schema as for byom predict
    # example (see README.md), we must add empty json_report column and change column order manually (v17.0.0.4)
    # CREATE MULTISET TABLE pima_patient_predictions
    # (
    #     job_id VARCHAR(255), -- comes from airflow on job execution
    #     PatientId BIGINT,    -- entity key as it is in the source data
    #     HasDiabetes BIGINT,   -- if model automatically extracts target
    #     json_report CLOB(1048544000) CHARACTER SET UNICODE  -- output of
    # )
    # PRIMARY INDEX ( job_id );
    predictions_pdf["json_report"] = ""
    predictions_pdf = predictions_pdf[["job_id", entity_key, target_name, "json_report"]]

    copy_to_sql(
        df=predictions_pdf,
        schema_name=context.dataset_info.predictions_database,
        table_name=context.dataset_info.predictions_table,
        index=False,
        if_exists="append"
    )
    
    print("Saved predictions in Teradata")

    # calculate stats
    predictions_df = DataFrame.from_query(f"""
        SELECT 
            * 
        FROM {context.dataset_info.get_predictions_metadata_fqtn()} 
            WHERE job_id = '{context.job_id}'
    """)

    record_scoring_stats(features_df=features_tdf, predicted_df=predictions_df, context=context)


In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the scoring dataset 

sql = f"""
SELECT TOP 100
*
FROM {Param['database']}.transactions_fraud
where fold = 'test'
"""

# where to store predictions
predictions = {
    "database": Param['database'],
    "table": "Transaction_fraud_predictions"
}

import uuid
job_id=str(uuid.uuid4())

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata,
                           predictions=predictions)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path=f'{model_local_path}/model_modules/artifacts/',
                   artifact_input_path=f'{model_local_path}/model_modules/artifacts/',
                   model_version="InDB_v1",
                   model_table="aoa_model_indb_v1",
                   job_id=job_id)

import scoring
scoring.score(context=ctx)
score(context=ctx)

## 5 - Define model metadata

In [ ]:
%%writefile "$model_local_path/model_modules/requirements.txt"
teradataml==17.20.0.3
aoa==7.0.1
pandas==1.1.5
scikit-learn==0.24.2
matplotlib==3.5.2

In [ ]:
%%writefile "$model_local_path/config.json"
{
   "hyperParameters": {
        "scale_method":"STD",
        "miss_value":"KEEP",
        "global_scale":"False",
        "multiplier":"1",
        "intercept":"0",
        "max_depth": 15, 
        "num_trees": 72,
        "min_node_size": 1,
        "mtry": 6,
        "mtry_seed": 1,
        "seed": 1
   }
}

In [ ]:
%%writefile "$model_local_path/model.json"
{
    "id": "f8df0bec-12d1-4d2d-920f-4448503df82e",
    "name": "Python Fraud Transaction InDB DecisionForest",
    "description": "Python InDB DecisionForest for Fraud Transaction prediction",
    "language": "python"
}